## Import dependancies and set environment determinism

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf
#     tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(
    0
)
import numpy as np
import random
import pandas as pd


SEED = 378
# SEED = 123
print(SEED)
def set_seeds(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)

def set_global_determinism(seed=SEED):
    set_seeds(seed=seed)

    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

# Call the above function with seed value
set_global_determinism(seed=SEED)

    


2024-01-11 18:02:48.278439: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


378


In [2]:
import glacierml as gl
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tqdm import tqdm

In [3]:
# if os.path.isdir(res_dir) == False:

def run_model(model_path,l1,l2,loss):
            
    normalizer = preprocessing.Normalization(axis=-1)
    normalizer.adapt(np.array(trfeat))

    model = gl.build_dnn_model(
        normalizer, learning_rate = 0.01, 
        layer_1 = l1, layer_2 = l2,loss = loss
    )

    model_history = model.fit(
        trfeat,
        trlabs,
        validation_split=0.2,
        callbacks = [callback],
        verbose=0, 
        epochs=500
    )
    model_filename = os.path.join(model_path)
    model.save(model_filename)
    return model

In [4]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    min_delta = 0.001,
    patience = 10,
    verbose = 0,
    mode = 'auto',
    baseline = None,
    restore_best_weights = True
)

In [5]:
for i in range(1,5,1):
    set_global_determinism(seed=SEED)
    df = gl.coregister_data(str(i))
    df = df.drop(df[df['RGIId'].duplicated(keep = False)].index)

    df = df.sample(frac = 1,random_state = 0)
    df = df.reset_index().drop('index', axis = 1)
    dft = df.copy()
#     df = df.drop(df[df['Area'] <= 0.05].index)
    df['Vol'] = (df['Thickness'] /1e3)  * (df['Area'])
    df = df.drop('Thickness',axis = 1)

    trfeat, tefeat, trlabs, telabs = gl.split_data(df,var = 'Vol')

    model = {}
    model_history = {}
    normalizer = {}
    model_path = os.path.join(
        '/data/fast1/glacierml/models/mse_coregistration_testing',str(i)
    )
    model = run_model(model_path,6,2,loss = 'mse')
    
    rgi_est_pth = os.path.join(model_path, 'rgi_est_raw.pkl')

    if os.path.isdir(rgi_est_pth) == False:

        RGI = gl.load_RGI()
        rfp = RGI[list(df)[:-1]]

        preds = pd.Series(
            model.predict(rfp.drop('RGIId',axis = 1)).flatten(), name = i
        )
        RGI = pd.concat([RGI,preds], axis = 1)
        RGI.to_pickle(rgi_est_pth)
                # RGI = pd.read_pickle('rgi_est_raw.pkl')

2024-01-11 18:02:50.458356: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


6766/6766 [==============================] - 5s 686us/step
